<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
<br>汉化的库: <a href="https://github.com/GoatCsu/CN-LLMs-from-scratch.git">https://github.com/GoatCsu/CN-LLMs-from-scratch.git</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 第二章:处理文本数据

原始仓库里ch02的README信息还有一个05 和一个video:

- 05_bpe-from-scratch 包含（额外）代码，用于从头实现和训练 GPT-2 BPE 分词器。

- 在下面的视频中，我提供了一个代码练习环节，其中涵盖了部分章节内容作为补充材料。
https://www.youtube.com/watch?v=341Rb8fJxY0


本章节需要安装的包

pip3 install importlib.metadata

In [19]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))
# 确认库已安装并显示当前安装的版本

torch version: 2.8.0+cu126
tiktoken version: 0.11.0


- 本章节已经为LLM的实现构建了数据库

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="500px">

## 2.1 理解文字embedding

- 无代码

- 在众多形式的embedding中,我们只讨论text embedding
- embedding含义丰富,而且是常用词汇,所以以下皆不做翻译,多加体会!

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">

- LLM从高纬空间视角理解文字(i.e., 上千个dimension)
- 虽然人类只能想象低维的视角,我们无法描绘计算机所理解的embedding
- 但是下图我们粗浅的从二维上模拟计算机的视角

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">

## 2.2 文本标签化(tokenize)

- 关于embedding token,实在是不好翻译,于是有时候我会选择不去翻译这两个专有名词
- 本节中,我们将tokenize文字信息. 这会把文字拆解为更多小的理解单元 例如单个词或者字节

(这也有点抽象，事实上可以粗略理解为将单词拆分为词根、词源和词缀)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="300px">

- 载入源文件
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict) 一本无版权的短篇小说

In [20]:
import os##导入os库
import urllib.request ##导入request库

if not os.path.exists("the-verdict.txt"):##如果文件不存在则创建，防止因文件已存在而报错
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)##从指定的地点读取文件

- 如果在执行前面的代码单元时遇到 `ssl.SSLCertVerificationError` 错误可能是由于使用了过时的 Python 版本；
- 你可以在 [GitHub 上查阅更多信息](https://github.com/rasbt/LLMs-from-scratch/pull/403)。

In [21]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read() ##读入文件按照utf-8

print("Total number of character:", len(raw_text))##先输出总长度
print(raw_text[:99])##输出前一百个内容

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 目标是对这段文本进行分词和嵌入处理，以便用于大语言模型（LLM）。
- 我们将基于一些简单的示例文本开发一个简单的分词器，之后可以将其应用于上述文本。
- 以下正则表达式将基于空格进行分割。

In [22]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)##正则表达式按照空白字符进行分割

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 优化正则表达,可以分割更多的符号

In [23]:
result = re.split(r'([,.]|\s)', text)##只是按照, .分割
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- 移除空格

In [24]:
##把上述结果去掉空格
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- 我们还需要处理其他标点符号

In [25]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text) ##就是按照常用的符号分割
result = [item.strip() for item in result if item.strip()]##去掉两端的空白字符 也是去掉了空字符串与仅包含空白字符的项
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- 万事俱备，我们现在来看一下文字处理后的效果

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">

In [26]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text) ##按照符号继续把原文件给分割了
preprocessed = [item.strip() for item in preprocessed if item.strip()]##去掉两端的空白字符 也是去掉了空字符串和仅包含空白字符的项
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- 查看token的长度

In [27]:
print(len(preprocessed))

4690


## 2.3 将词元转换为词元 ID (给token编号)

- 通过如下的embedding层,我们可以给token编号
- 把这些词元从 Python 字符串转换为整数表示，以生成词元 ID（token ID）。这一过程是将词元 ID 转换为嵌入向量前的必经步骤

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">

- 为了将先前生成的词元映射到词元 ID，首先需要构建一张词汇表。这张词汇表定义了如何将每个唯一的单词和特殊字符映射到一个唯一的整数

In [28]:
all_words = sorted(set(preprocessed))#从去掉重复的字符
vocab_size = len(all_words)#计总的单词书

print(vocab_size)

1130


In [29]:
vocab = {token:integer for integer,token in enumerate(all_words)}##先把word进行编号,再按照单词或者标点为索引(有HashList那味道了)

- 看一下前50个是怎样的

In [30]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break ##遍历到前五十个

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- 接下来,我们将通过一个短文本来感受下,处理后的效果是怎样的

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="500px">

- 现在将所有内容整合到一个分词器类中。

In [31]:
class SimpleTokenizerV1:#一个实例的名字创立
    def __init__(self, vocab): ## 初始化一个字符串
        self.str_to_int = vocab #单词到整数的映射, 将词汇表作为类属性存储，以便在 encode方法和 decode 方法中访问
        self.int_to_str = {i:s for s,i in vocab.items()} #创建逆向词汇表，将词元 ID 映射回原始文本词元
        #方便解码,进行整数到词汇的反向映射

    def encode(self, text):
        '''
        处理输入文本，将其转换为词元 ID
        '''
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)##正则化分词标点符号

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()## 去掉两端空格与全部的空句
        ]
        ids = [self.str_to_int[s] for s in preprocessed]##整理完的额字符串列表对应到id,从字典出来
        return ids

    def decode(self, ids): #将词元ID转换回文本
        text = " ".join([self.int_to_str[i] for i in ids]) #映射整数id到字符串。join是用前面那个(“ ”)联结成一个完整的字符串
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) #使用正则表达式，去除标点符号前的多余空格
        # \s+匹配一个或者多个空白  \1 替换到匹配
        return text

`SimpleTokenizerV1` 的简单分词器类。这个类的主要作用是将文本转换为数字（词元 ID），以及将数字（词元 ID）转换回文本。

以下是代码的详细解释：

- **`class SimpleTokenizerV1:`**: 定义了一个名为 `SimpleTokenizerV1` 的类。
- **`__init__(self, vocab)`**: 这是类的构造函数，在创建 `SimpleTokenizerV1` 对象时被调用。
    - `self.str_to_int = vocab`: 将输入的 `vocab` 字典（用于将字符串词元映射到整数 ID）存储为类的属性 `str_to_int`。
    - `self.int_to_str = {i:s for s,i in vocab.items()}`: 创建一个逆向字典 `int_to_str`，用于将整数 ID 映射回字符串词元。
- **`encode(self, text)`**: 这个方法用于将输入的文本字符串编码为词元 ID 的列表。
    - `preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)`: 使用正则表达式按照标点符号和空白字符分割输入的文本。
    - `preprocessed = [item.strip() for item in preprocessed if item.strip()]`: 遍历分割后的结果，去除每个词元两端的空白字符，并过滤掉只包含空白字符的项，得到一个干净的词元列表。
    - `ids = [self.str_to_int[s] for s in preprocessed]`: 遍历处理后的词元列表，使用 `str_to_int` 字典将每个词元映射到其对应的整数 ID，生成一个整数 ID 的列表。
    - `return ids`: 返回生成的词元 ID 列表。
- **`decode(self, ids)`**: 这个方法用于将输入的词元 ID 列表解码回文本字符串。
    - `text = " ".join([self.int_to_str[i] for i in ids])`: 遍历输入的词元 ID 列表，使用 `int_to_str` 字典将每个 ID 映射回其对应的字符串词元，然后使用空格将这些词元连接成一个字符串。
    - `text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)`: 使用正则表达式去除标点符号前多余的空格，使解码后的文本更自然。`\s+` 匹配一个或多个空白字符，`([,.?!"()\'])` 捕获一个标点符号，`\1` 表示替换为捕获到的标点符号本身。
    - `return text`: 返回解码后的文本字符串。

总的来说，`SimpleTokenizerV1` 类提供了一个基本的分词和反分词功能，它依赖于一个预先构建好的词汇表来将词元和整数 ID 进行相互转换。

- `encode` 函数将文本转换为标记 ID。
- `decode` 函数将标记 ID 转换回文本。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">

- 我们可以使用分词器将文本编码（即分词）为数字。
- 然后，这些整数可以作为大语言模型（LLM）的输入，进行嵌入。
- 分词器通常包含两个常见的方法：encode 方法和 decode 方法。encode 方法接收文本样本，将其分词为单独的词元，然后再利用词汇表将词元转换为词元 ID。而 decode 方法接收一组词元 ID，将其转换回文本词元，并将文本词元连接起来，形成自然语言文本

In [32]:
tokenizer = SimpleTokenizerV1(vocab) #用vocab创造一个实例

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text) #按照这个例子里的encode函数处理text
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- 把数字重新映射回文字

In [33]:
tokenizer.decode(ids)#按照这个例子里的decode函数处理text

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [34]:
tokenizer.decode(tokenizer.encode(text))#按照这个例子里的decode函数处理(#按照这个例子里的encode函数处理text)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 添加特殊标记

- 文本的结尾需要特别的符号来表明

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">

- 一些分词器使用特殊标记来帮助大语言模型（LLM）获取额外的上下文信息。
- 其中一些特殊标记包括：
  - `[BOS]`（序列开始）表示文本的开始。
  - `[EOS]`（序列结束）表示文本的结束（通常用于连接多个不相关的文本，例如两个不同的维基百科文章或两本不同的书籍等）。
  - `[PAD]`（填充）如果我们使用大于1的批次大小训练LLM（我们可能会包含不同长度的多篇文本），使用填充标记将较短的文本填充至最长的长度，以确保所有文本具有相同的长度。
- `[UNK]` 用于表示词汇表中没有的词。

- 请注意，GPT-2不需要上述提到的这些标记，它只使用 `<|endoftext|>` 标记。
- `<|endoftext|>` 类似于上述提到的 `[EOS]` 标记。
- GPT 还使用 `<|endoftext|>` 进行填充（因为我们在批量输入训练时通常使用掩码，所以无论这些填充标记是什么，都不会影响模型的训练，因为填充标记不会被关注）。
- GPT-2 不使用 `<UNK>` 标记来表示词汇表外的词；相反，GPT-2 使用字节对编码（BPE）分词器，将单词分解为子词单元，后续将进一步讨论这一点。
- 我们在两个独立文本之间使用 `<|endoftext|>` 标记：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">

- 看一下接下来会发生什么

In [35]:
tokenizer = SimpleTokenizerV1(vocab)  ##用vocab创造一个实例

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

- 上述操作会产生一个错误，因为单词“Hello”不在词汇表中。
- 为了处理这种情况，我们可以向词汇表中添加类似 `"<|unk|>"` 的特殊标记，用于表示未知词汇。
- 因为我们已经在扩展词汇表，那么我们可以再添加一个标记 `"<|endoftext|>"`，该标记在GPT-2训练中用于表示文本的结束（它也用于连接的文本之间，例如当我们的训练数据集包含多篇文章、书籍等时）。

In [36]:
all_tokens = sorted(list(set(preprocessed)))#set去重 list把处理后的重新变为列表,然后排序
all_tokens.extend(["<|endoftext|>", "<|unk|>"])#加上未知的表示

vocab = {token:integer for integer,token in enumerate(all_tokens)}
#遍历 enumerate(all_tokens) 中的每个元组 (integer, token)，以 token 作为键，integer 作为值创建字典条目。

In [37]:
len(vocab.items())

1132

In [38]:
for i, item in enumerate(list(vocab.items())[-5:]):#输出后五个内容与其标号
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- 因此增加 `<unk>`不失为一种好的选择

In [39]:
class SimpleTokenizerV2:##版本2.0,启动!
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}#s为单词,i是key

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)#正则化按照标点分类
        preprocessed = [item.strip() for item in preprocessed if item.strip()]#去掉两头与所有空余句
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
            #遍历 preprocessed 中的每个 item，如果 item 存在于 self.str_to_int（即词汇表）中，就保留 item
            #如果不存在（即该单词或符号未定义在词汇表中），就替换为特殊标记 <|unk|>。
            #拓展:推导式（如列表推导式）是一种紧凑的语法，专门用于生成新列表（或其他容器）
            #与普通 for 循环相比，它更加简洁和高效，但逻辑复杂时可能会降低可读性。
        ]

        ids = [self.str_to_int[s] for s in preprocessed]#单词或标点映射为整数列表
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

- 用优化后的分词器对文本进行操作

In [40]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))#用句子分隔符链接两个句子

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [41]:
tokenizer = SimpleTokenizerV2(vocab)
tokenizer.encode(text)#跟第一个一样,但不会报错了

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [42]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 字节对编码

- GPT-2 使用字节对编码（BPE）作为其分词器。
- 这种方式允许模型将不在预定义词汇表中的单词分解为更小的子词单元，甚至是单个字符，从而使其能够处理词汇表外的词汇。
- 例如，如果 GPT-2 的词汇表中没有“unfamiliarword”这个单词，它可能会将其分词为 ["unfam", "iliar", "word"]，或者根据训练好的 BPE 合并规则进行其他子词分解。
- 原始的 BPE 分词器可以在这里找到：[https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- 在本章中，我们使用了 OpenAI 开源的 [tiktoken](https://github.com/openai/tiktoken) 库中的 BPE 分词器，该库在 Rust 中实现了核心算法，以提高计算性能。
- 在 [./bytepair_encoder](../02_bonus_bytepair-encoder) 中创建了一个笔记本，对比了这两种实现的效果（在样本文本上，tiktoken 的速度大约快了 5 倍）。

In [43]:
# pip install tiktoken

In [44]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))#验证下载并输出版本信息

tiktoken version: 0.11.0


In [45]:
tokenizer = tiktoken.get_encoding("gpt2")#初始化GPT2! 按照以下方式实例化 tiktoken 中的 BPE 分词器

In [46]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})#输出分词的id,可以允许endoftext

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


1. <|endoftext|>词元被分配了一个较大的词元 ID，即 50256。事实上，用于训练GPT-2、GPT-3 和 ChatGPT 中使用的原始模型的 BPE 分词器的词汇总量为 50 257，这意味着
<|endoftext|>被分配了最大的词元 ID。
2. BPE 分词器可以正确地编码和解码未知单词，比如“someunknownPlace”。BPE 分词器是如何做到在不使用<|unk|>词元的前提下处理任何未知词汇的呢？
- BPE 算法的原理是将不在预定义词汇表中的单词分解为更小的子词单元甚至单个字符，从而能够处理词汇表之外的单词。因此，得益于 BPE 算法，如果分词器在分词过程中遇到不熟悉的单词，它可以将其表示为子词词元或字符序列

- BPE tokenizers将未知词汇分解为子词和单个字符。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="300px">

In [47]:
text_testing = ("Akwirwier")

integers_testing = tokenizer.encode(text_testing, allowed_special={"<|endoftext|>"})

print("词元ID: ",integers_testing)

strings_testing = tokenizer.decode(integers_testing)

print("词元: ",strings_testing)

词元ID:  [33901, 86, 343, 86, 959]
词元:  Akwirwier


## 2.6 利用滑动窗口进行数据采样

- 现在我们训练的大语言模型（LLMs）时是一次生成一个单词，因此希望根据训练数据的要求进行准备，使得序列中的下一个单词作为预测目标。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="400px">

实现一个数据加载器，使用**滑动窗口**（sliding window）方法从训练数据集中提取,**首先**，使用 BPE 分词器对短篇小说 The Verdict 的全文进行分词

In [48]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)#读入了一个text并编码到enc_text里面
print(len(enc_text)) #是应用 BPE 分词器后训练集中的词元总数

5145


- 对于每个文本块，我们需要输入和目标。
- 由于我们希望模型预测下一个单词，因此我们要生成目标是将输入右移一个位置后的单词。

In [49]:
enc_sample = enc_text[50:]#从第五十一个开始向后,从数据集中移除前 50 个词元以便演示

创建下一单词预测任务的输入-目标对的一种简单且直观的方法是定义两个变量：x 和 y。变量 x 用于存储输入的词元，变量 y 则用于存储由 x 的每个输入词元右移一个位置所得的目标词元

In [50]:
context_size = 4 #sliding windows4

x = enc_sample[:context_size]#开头四个
y = enc_sample[1:context_size+1]#第二个开始的四个

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- 就像预言家一个晚上只能预言一个玩家,我们的模型一次也只能预测一个单词

In [51]:
for i in range(1, context_size+1):
    #文本成输入 context,先输出有什么,然后输出下一个是什么编号
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [52]:
for i in range(1, context_size+1):
    #文本成输入 context,先输出有什么,然后输出下一个是什么单词
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- 我们将在后续章节中处理下一个单词预测，届时会介绍注意力机制。
- 目前，我们实现一个简单的数据加载器，它遍历输入数据集并返回右移一个位置后的输入和目标。

- 安装并导入 PyTorch（安装提示请参见附录A）。

In [53]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.8.0+cu126


- 用滑动窗口法运行,窗口位置每次加一

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">

- 创建数据集和数据加载器，从输入文本数据集中提取文本块。

一个用于批处理输入和目标的数据集

In [54]:
from torch.utils.data import Dataset, DataLoader
#Dataset 是 PyTorch 中用于表示数据集的抽象类，DataLoader 则用于高效地加载和批量处理数据

class GPTDatasetV1(Dataset):#定义了一个继承自 Dataset 的新类 GPTDatasetV1
    #让GPT初始化一个类型
    def __init__(self, txt, tokenizer, max_length, stride):# 这是类的构造函数，用于初始化数据集
        self.input_ids = [] #初始化两个空列表，分别用于存储输入的词元 ID 序列
        self.target_ids = [] #目标词元 ID 序列

        # 对全部文本进行分词 -使用传入的 tokenizer 对输入的文本 txt 进行编码，将文本转换为词元 ID 列表
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})#id是文本内容编码过来的
        #允许分词器处理 <|endoftext|> 特殊标记

        # 使用滑动窗口将文本划分为长度为 max_length 的重叠序列
        for i in range(0, len(token_ids) - max_length, stride): #使用滑动窗口的方式从 token_ids 中提取序列
            #从 0 开始，到 len(token_ids) - max_length 结束（不包含），步长为 stride 的整数序列
            input_chunk = token_ids[i:i + max_length] #提取从索引 i 开始，长度为 max_length 的词元 ID 序列作为输入
            target_chunk = token_ids[i + 1: i + max_length + 1] #提取从索引 i + 1 开始，长度为 max_length 的词元 ID 序列作为目标。这实现了下一个词预测的任务，目标序列是输入序列向右移动一个位置的结果
            self.input_ids.append(torch.tensor(input_chunk)) #将 input_chunk 转换为 PyTorch 张量并添加到 self.input_ids 列表中
            self.target_ids.append(torch.tensor(target_chunk)) #将 target_chunk 转换为 PyTorch 张量并添加到 self.target_ids 列表中

    def __len__(self): #这是 Dataset 类必须实现的方法，返回数据集中的样本数量, 即返回数据集的总行数
        return len(self.input_ids) #在这里，它返回 self.input_ids 列表的长度，因为每个输入序列都有一个对应的目标序列

    def __getitem__(self, idx): #这是 Dataset 类必须实现的方法，用于根据索引 idx 获取数据集中的一个样本, 即返回数据集的指定行
        return self.input_ids[idx], self.target_ids[idx] #返回 self.input_ids 中索引为 idx 的输入序列和 self.target_ids 中索引为 idx 的目标序列

#stride 参数控制了窗口的重叠程度，较大的 stride 会减少样本数量和重叠，较小的 stride 会增加样本数量和重叠

用于批量生成输入-目标对的数据加载器

In [55]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # 初始化分词器 -初始化一个 GPT-2 的分词器，使用 tiktoken 库
    tokenizer = tiktoken.get_encoding("gpt2")

    # 创建数据集
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride) #这个对象包含了处理后的输入和目标序列

    # 创建 dataloader 对象
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last, #如果 drop_last 为 True 且批次大小小于指定的 batch_size，则会删除最后一批，以防止在训练期间出现损失剧增
        num_workers=num_workers #用于预处理的 CPU 进程数
    )

    return dataloader

**函数参数意思**:
- txt: 输入的原始文本数据。
- batch_size=4: 每个批次包含的样本数量，默认为 4。
- max_length=256: 每个输入序列的最大长度，默认为 256。
- stride=128: 滑动窗口的步长，默认为 128。
- shuffle=True: 是否在每个 epoch 开始时打乱数据，默认为 True。
- drop_last=True: 如果数据集大小不能被批次大小整除，是否丢弃最后一个不完整的批次，默认为 True。
- num_workers=0: 用于数据加载的子进程数量，默认为 0（表示在主进程中加载数据）。

**dataloader = DataLoader(...)参数意思:**
- dataset: 指定要加载的数据集对象，即前面创建的 GPTDatasetV1 实例。
- batch_size=batch_size: 设置每个批次的样本数量。
- shuffle=shuffle: 设置是否打乱数据。
- drop_last=drop_last: 设置是否丢弃最后一个不完整的批次。
- num_workers=num_workers: 设置用于数据加载的子进程数量。

- 让我们使用批次大小为1、上下文大小为4的设置，测试数据加载器在大语言模型（LLM）中的表现。

In [56]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [57]:
dataloader = create_dataloader_v1(#raw_text 中创建一个数据加载器 但是所批次
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)#将 dataloader 转换为 Python 迭代器，以通过 Python 内置的 next()函数获取下一个条目
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [58]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- 下面是一个示例，步幅等于上下文长度（此处为4）：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">

- 我们还可以批量输出。
- 因为过多的重叠可能导致过拟合,这里增加了步幅。

为了更直观地了解数据加载器的工作原理，请尝试使用不同的参数设置来运行它，比如max_length=2, stride=2 和 max_length=8, stride=2

In [80]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=2, stride=2, shuffle=False
)

for i, batch in enumerate(dataloader):
    print(f"Batch {i}: {batch}")
    if i ==5:
      break

Batch 0: [tensor([[ 40, 367]]), tensor([[ 367, 2885]])]
Batch 1: [tensor([[2885, 1464]]), tensor([[1464, 1807]])]
Batch 2: [tensor([[1807, 3619]]), tensor([[3619,  402]])]
Batch 3: [tensor([[402, 271]]), tensor([[  271, 10899]])]
Batch 4: [tensor([[10899,  2138]]), tensor([[2138,  257]])]
Batch 5: [tensor([[ 257, 7026]]), tensor([[ 7026, 15632]])]


In [81]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=8, stride=2, shuffle=False
)

for i, batch in enumerate(dataloader):
    print(f"Batch {i}: {batch}")
    if i ==5:
      break

Batch 0: [tensor([[  40,  367, 2885, 1464, 1807, 3619,  402,  271]]), tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899]])]
Batch 1: [tensor([[ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138]]), tensor([[ 1464,  1807,  3619,   402,   271, 10899,  2138,   257]])]
Batch 2: [tensor([[ 1807,  3619,   402,   271, 10899,  2138,   257,  7026]]), tensor([[ 3619,   402,   271, 10899,  2138,   257,  7026, 15632]])]
Batch 3: [tensor([[  402,   271, 10899,  2138,   257,  7026, 15632,   438]]), tensor([[  271, 10899,  2138,   257,  7026, 15632,   438,  2016]])]
Batch 4: [tensor([[10899,  2138,   257,  7026, 15632,   438,  2016,   257]]), tensor([[ 2138,   257,  7026, 15632,   438,  2016,   257,   922]])]
Batch 5: [tensor([[  257,  7026, 15632,   438,  2016,   257,   922,  5891]]), tensor([[ 7026, 15632,   438,  2016,   257,   922,  5891,  1576]])]


In [90]:
dataloader = create_dataloader_v1(raw_text, batch_size=2, max_length=8, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464,  1807,  3619,   402,   271],
        [ 1807,  3619,   402,   271, 10899,  2138,   257,  7026]])

Targets:
 tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899],
        [ 3619,   402,   271, 10899,  2138,   257,  7026, 15632]])


In [91]:
inputs1, targets1 = next(data_iter)
print("2nd Inputs:\n", inputs1)
print("\n 2nd Targets:\n", targets1)

2nd Inputs:
 tensor([[10899,  2138,   257,  7026, 15632,   438,  2016,   257],
        [15632,   438,  2016,   257,   922,  5891,  1576,   438]])

 2nd Targets:
 tensor([[ 2138,   257,  7026, 15632,   438,  2016,   257,   922],
        [  438,  2016,   257,   922,  5891,  1576,   438,   568]])


## 2.7 创建标记嵌入(Creating token embeddings)

- 数据库快被准备好用于大语言模型（LLM）。
- 最后，让我们使用嵌入层将标记转换为连续的向量表示。
- 通常，这些嵌入层是大语言模型的一部分，并在模型训练过程中进行更新（训练）。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="400px">

大语言模型的输入文本的准备工作包括文本分词、将词元转换为词元 ID，以及将词元ID 转换为嵌入向量。本节将利用此前生成的词元 ID 来创建词元嵌入向量

由于类 GPT 大语言模型是使用**反向传播算法**（backpropagation algorithm）训练的深度神经网络，因此需要连续的向量表示或嵌入

- 假设我们有以下四个输入示例，经过分词后对应的输入 ID 为 2、3、5 和 1：

In [60]:
input_ids = torch.tensor([2, 3, 5, 1])#要加入2,3,5,1的字符

- 为了简化问题，假设我们只有一个包含 6 个词的小型词汇表，并且我们希望创建大小为 3 的嵌入。

In [61]:
vocab_size = 6#嵌入层需要支持的唯一标记的总数
output_dim = 3#嵌入向量的维度

torch.manual_seed(123)#用于设置随机数生成器的种子，确保结果的可复现性
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)#每行表示一个标记的嵌入向量。

- 结果是个6*3的矩阵

In [62]:
print(embedding_layer.weight) #其中每一行对应词汇表中的一个词元，每一列则对应一个嵌入维度

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- 对于熟悉（one-hot encoding）的人来说，上述嵌入层方法本质上只是实现one-hot编码后接矩阵乘法的更高效方式，具体实现可以参考[./embedding_vs_matmul](../03_bonus_embedding-vs-matmul)中的补充代码。
- 嵌入层只是对one-hot encoding和矩阵乘法方法的高效实现，因此可以将其视为一个神经网络层，并通过反向传播进行优化。

- 通过下列操作,我们可以把id3映射到一个三纬的矩阵

In [63]:
print(embedding_layer(torch.tensor([3]))) #现在将其应用到一个词元 ID 上，以获取嵌入向量

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- 请注意，上述内容是 `embedding_layer` 权重矩阵中的第四行。
- 要嵌入上述所有四个 `input_ids` 值，我们执行以下操作：

In [64]:
print(embedding_layer(input_ids)) #打印的输出显示，结果是一个 4×3 的矩阵

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- 嵌入层本质上是一个查找操作：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

- **您可能对比较嵌入层与常规线性层的附加内容感兴趣：[../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)**

In [92]:
print(embedding_layer(input_ids).shape)

torch.Size([4, 3])


`embedding_layer = torch.nn.Embedding(vocab_size, output_dim)` 这行代码创建了一个 PyTorch 的嵌入层。

- `torch.nn.Embedding`: 这是 PyTorch 中用于创建嵌入查找表的类。
- `vocab_size`: 表示词汇表的大小，也就是嵌入层需要支持的唯一词元（token）的总数。在这个例子中，`vocab_size = 6`，表示词汇表中有 6 个不同的词元。
- `output_dim`: 表示每个词元将被映射到的嵌入向量的维度。在这个例子中，`output_dim = 3`，表示每个词元将由一个 3 维的向量表示。

所以，`embedding_layer = torch.nn.Embedding(vocab_size, output_dim)` 创建了一个嵌入层，它可以将词汇表中 `vocab_size` 个不同的词元映射到 `output_dim` 维的向量空间中。

`embedding_layer.weight` 是什么意思？

- `embedding_layer.weight`: 这是嵌入层内部的一个可学习参数，它是一个形状为 `(vocab_size, output_dim)` 的权重矩阵。这个矩阵的每一行对应词汇表中的一个词元，每一列则对应嵌入向量的一个维度。
- 当您打印 `embedding_layer.weight` 时，您看到的就是这个权重矩阵的当前值。这些值在模型训练过程中会通过反向传播进行更新和优化，以便学习到更好的词元表示。

`embedding_layer(torch.tensor([3]))` 是什么意思？

- `embedding_layer(...)`: 当您像函数一样调用 `embedding_layer` 并传入一个包含词元 ID 的 PyTorch 张量时，嵌入层会执行一个查找操作。
- `torch.tensor([3])`: 这是一个包含单个词元 ID `3` 的 PyTorch 张量。
- `embedding_layer(torch.tensor([3]))`: 这表示您想查找词汇表中 ID 为 `3` 的词元对应的嵌入向量。
- 输出结果是一个形状为 `(1, output_dim)` 的张量，其中 `1` 表示输入的批次大小（这里是单个词元），`output_dim` 是嵌入向量的维度。这个张量包含了 ID 为 `3` 的词元在 `embedding_layer.weight` 矩阵中对应的行向量。

本质上，`embedding_layer` 的调用操作就是根据输入的词元 ID，在 `embedding_layer.weight` 矩阵中查找并取出对应的行向量，作为该词元的嵌入表示。这是一种高效的查找操作，可以视为 one-hot 编码后与权重矩阵相乘的简化和优化。

如果您传入一个包含多个词元 ID 的张量，例如 `embedding_layer(torch.tensor([2, 3, 5, 1]))`，嵌入层将返回一个形状为 `(batch_size, sequence_length, output_dim)` 的张量，其中 `batch_size` 是输入 ID 张量的第一个维度大小，`sequence_length` 是输入 ID 张量的第二个维度大小（如果存在），`output_dim` 是嵌入向量的维度。这个张量包含了输入中每个词元对应的嵌入向量。

**最后一段的理解**:
好的，您在使用 embedding_layer(input_ids).shape 时看到了 torch.Size([4, 3]) 的输出，对我的解释感到困惑是正常的。

让我详细解释一下：

您观察到的 `torch.Size([4, 3])` 是完全正确的，这与我之前的解释是吻合的。

我们回顾一下您使用的代码：

```python
input_ids = torch.tensor([2, 3, 5, 1])
embedding_layer = torch.nn.Embedding(vocab_size, output_dim) # vocab_size=6, output_dim=3
print(embedding_layer(input_ids).shape)
在这里：

input_ids = torch.tensor([2, 3, 5, 1]) 创建了一个形状为 (4,) 的一维张量。它包含了 4 个词元 ID。
当您将这个形状为 (4,) 的一维张量传递给 embedding_layer 时，嵌入层会查找并取出这 4 个 ID 对应的嵌入向量。
每个嵌入向量的维度是 output_dim，也就是 3。
因此，输出的张量包含了 4 个形状为 (3,) 的向量，它们被堆叠在一起，形成了形状为 (4, 3) 的二维张量。
我的解释中提到的形状 (batch_size, sequence_length, output_dim) 是更一般的情况，特别是当您的输入 input_ids 是一个批次（batch）的数据时。

想象一下，如果您有一个包含多个句子（或序列）的批次，每个句子都由一系列词元 ID 组成。在这种情况下，您的输入 input_ids 会是一个形状为 (batch_size, sequence_length) 的二维张量，其中：

batch_size 是批次中句子的数量。
sequence_length 是每个句子的长度（包含的词元数量）。
当您将这样一个形状为 (batch_size, sequence_length) 的二维张量传递给嵌入层时，嵌入层将为批次中的每个句子、每个位置的词元 ID 查找对应的嵌入向量。输出的张量将是形状为 (batch_size, sequence_length, output_dim) 的三维张量，其中：

第一个维度是 batch_size，对应批次中的每个句子。
第二个维度是 sequence_length，对应句子中的每个位置。
第三个维度是 output_dim，对应每个词元的嵌入向量维度。
在您提供的示例中，您的输入 input_ids 是一个形状为 (4,) 的一维张量。在这种情况下，我们可以将这个一维张量视为一个批次大小为 4，序列长度为 1 的特殊情况，或者更直观地理解为，它是一个包含 4 个独立词元 ID 的序列。嵌入层为这 4 个词元 ID 分别查找了对应的 3 维嵌入向量，所以输出的形状是 (4, 3)。

简单来说：

如果输入是形状为 (N,) 的一维张量，输出将是形状为 (N, output_dim) 的二维张量。
如果输入是形状为 (N, L) 的二维张量（表示一个包含 N 个序列的批次，每个序列长度为 L），输出将是形状为 (N, L, output_dim) 的三维张量。
您的 input_ids 是一个一维张量，所以输出是二维的 (4, 3)。我的解释是针对更常见的、包含多个序列的批次输入，所以给出了三维的形状 (batch_size, sequence_length, output_dim)。

希望这次解释更清晰了！

## 2.8 位置信息编码

- 嵌入层将 ID 转换为相同的向量表示

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="400px">

- 位置信息与标记向量结合，形成大语言模型的最终输入：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

- 字节对编码器的词汇表大小为 50,257：
- 假设我们想将输入标记编码为一个 256 维的向量表示：

In [65]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)#映射为tensor

- 如果我们从数据加载器中采样数据，我们将每个批次中的标记嵌入为一个 256 维的向量。
- 如果我们有一个批次大小为 8，每个批次包含 4 个标记，这将得到一个 8 x 4 x 256 的张量：

In [66]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [67]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [68]:
token_embeddings = token_embedding_layer(inputs)#调用token_embedding_layer将输入inputs映射为对应的嵌入向量。
print(token_embeddings.shape)

torch.Size([8, 4, 256])


- GPT-2 使用绝对位置嵌入，因此我们只需要创建另一个位置嵌入层：

In [69]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
#目的是为输入序列中的每个位置生成一个向量,表明位置信息

- 嵌入层本质上是一个查找表,大小为(context_length, output_dim)

In [70]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))#生成一个连续整数的1D tensor

print(pos_embeddings.shape)

torch.Size([4, 256])


- 为了创建大语言模型（LLM）中使用的输入嵌入，我们只需将标记嵌入和位置嵌入相加：

In [71]:
input_embeddings = token_embeddings + pos_embeddings#特征是词语信息跟位置信息的结合
print(input_embeddings.shape)

torch.Size([8, 4, 256])


- 在输入处理流程的初始阶段，输入文本被分割为独立的标记。
- 在此分割后，这些标记根据预定义的单词表转换为标记 ID：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">

# 总结与收获

请参见 [./dataloader.ipynb](./dataloader.ipynb) 代码笔记本，这是我们在本章中实现的数据加载器的简洁版，并将在后续章节中用于训练 GPT 模型。

请参见 [./exercise-solutions.ipynb](./exercise-solutions.ipynb) 获取习题解答。